In [ ]:
# Generate samples for 11 combinations of tasks on CIFAR-10
!python scripts/generate_samples.py --task=sr_4x_noisy --save_dir=./logs/cifar/samples_sr_4x_noisy_poisson --noise_type=poisson
!python scripts/generate_samples.py --task=sr_4x_noisy --save_dir=./logs/cifar/samples_sr_4x_noisy_gaussian --noise_type=gaussian
!python scripts/generate_samples.py --task=sr_4x_noisy --save_dir=./logs/cifar/samples_sr_4x_noisy_poisson_opti --lambda_=0.045 --noise_type=poisson
!python scripts/generate_samples.py --task=sr_4x_noisy --save_dir=./logs/cifar/samples_sr_4x_noisy_gaussian_opti --lambda_=0.045 --noise_type=gaussian
!python scripts/generate_samples.py --task=deblur_colorise --save_dir=./logs/cifar/samples_deblur_colorise
!python scripts/generate_samples.py --task=denoise_colorise --save_dir=./logs/cifar/samples_denoise_colorise_opti --lambda_=0.045
!python scripts/generate_samples.py --task=denoise_colorise --save_dir=./logs/cifar/samples_denoise_colorise
!python scripts/generate_samples.py --task=sr_4x_colorise --save_dir=./logs/cifar/samples_sr_4x_colorise
!python scripts/generate_samples.py --task=sr_4x_deblur --save_dir=./logs/cifar/samples_sr_4x_deblur
!python scripts/generate_samples.py --task=denoise_deblur --save_dir=./logs/cifar/samples_denoise_deblur
!python scripts/generate_samples.py --task=denoise_deblur --save_dir=./logs/cifar/samples_denoise_deblur_opti --lambda_=0.045

In [ ]:
!python scripts/compute_combined_metrics.py

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

individual_df = pd.read_csv('metrics_output_individual.csv')
aggregated_df = pd.read_csv('metrics_output_aggregated.csv')

# Rename tasks to A, B, C, D, E, F, G, H, I, J, K
task_rename_dict = {task: f'Task {chr(65+i)}' for i, task in enumerate(aggregated_df['task_name'].unique())}
print(task_rename_dict)
aggregated_df['task_name'] = aggregated_df['task_name'].map(task_rename_dict)
individual_df['task_name'] = individual_df['task_name'].map(task_rename_dict)

# Visualization for Individual Metrics
plt.figure(figsize=(18, 6))
for i, metric in enumerate(['ssim', 'psnr', 'lpips']):
    plt.subplot(1, 3, i+1)
    sns.scatterplot(data=individual_df, x='task_name', y=metric)
    plt.title(f'Individual {metric.upper()} values')
    plt.ylabel(f'{metric.upper()} Value')
    plt.xlabel('Task')
plt.tight_layout()
plt.show()

# Visualization for Aggregated Metrics (Bootstrap and Statistical)
for metric in ['ssim', 'psnr', 'lpips']:
    metric_df = aggregated_df[aggregated_df['metric'] == metric]
    plt.figure(figsize=(15, 5))
    plt.suptitle(f'Aggregated {metric.upper()} Metrics with Confidence Intervals')

    # Bootstrap Aggregated Metrics
    plt.subplot(1, 2, 1)
    sns.barplot(x='task_name', y='bootstrap_mean', data=metric_df, color='skyblue')
    plt.errorbar(x=range(len(metric_df)), y=metric_df['bootstrap_mean'],
                 yerr=(metric_df['bootstrap_CI_upper'] - metric_df['bootstrap_CI_lower']) / 2,
                 fmt='none', capsize=5, color='blue', label='95% CI (Bootstrap)')
    plt.title('Bootstrap Aggregated Metrics')
    plt.ylabel(f'{metric.upper()} Value')
    plt.xlabel('Task')
    plt.xticks(range(len(metric_df)), labels=metric_df['task_name'], rotation=45)
    plt.legend()

    # Statistical Aggregated Metrics
    plt.subplot(1, 2, 2)
    sns.barplot(x='task_name', y='mean', data=metric_df, color='salmon')
    plt.errorbar(x=range(len(metric_df)), y=metric_df['mean'],
                 yerr=(metric_df['statistical_CI_upper'] - metric_df['statistical_CI_lower']) / 2,
                 fmt='none', capsize=5, color='green', label='95% CI (Statistical)')
    plt.title('Statistical Aggregated Metrics')
    plt.ylabel(f'{metric.upper()} Value')
    plt.xlabel('Task')
    plt.xticks(range(len(metric_df)), labels=metric_df['task_name'], rotation=45)
    plt.legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()